In [ ]:
import numpy as np
import os

os.chdir((os.getcwd()+os.sep+"mnisttxt"))
#Load train & test datasets
for digit in range(10):
    if (digit == 0):
        train = np.loadtxt("train"+str(digit)+".txt",dtype=np.float64)
        labels = np.full((train.shape[0], 1), digit,dtype = np.float64)
        train = np.concatenate((train, labels), axis=1)
        test = np.loadtxt("test"+str(digit)+".txt",dtype=np.float64)
        test_labels = np.full((test.shape[0], 1), digit,dtype =np.float64)
        test = np.concatenate((test, test_labels), axis=1)
    else:
        train2 = np.loadtxt("train"+str(digit)+".txt",dtype=np.float64)
        labels = np.full((train2.shape[0], 1), digit,dtype =np.float64)
        train2 = np.concatenate((train2, labels), axis=1)
        train = np.concatenate((train, train2), axis=0)
        test2 = np.loadtxt("test"+str(digit)+".txt",dtype=np.float64)
        test_labels = np.full((test2.shape[0], 1), digit,dtype = np.float64)
        test2 = np.concatenate((test2, test_labels), axis=1)
        test = np.concatenate((test, test2), axis=0)

#Shuffle both train and test datasets
np.random.seed(1234)
np.random.shuffle(train)
np.random.shuffle(test)


#Split train,test data and labels
train_data = train[:,:784]
train_labels = train[:,784]
train_labels = train_labels.astype('int')

test_data = test[:,:784]
test_labels = test[:,784]
test_labels = test_labels.astype('int')


# min-max scale of train and test data
np.seterr(all='ignore') # disable possible warnings for 0 divisions
X_min = np.min(train_data,axis = 0)
X_max = np.max(train_data,axis = 0)
max_min = np.subtract(X_max,X_min)
train_data = (train_data - X_min) / max_min
test_data = (test_data - X_min) / max_min
#replace nans(from possible zero divisions) with 0
train_data[np.isnan(train_data)] = 0
test_data[np.isnan(test_data)] = 0

In [ ]:
#plot the 10 first digits of train dataset
%matplotlib inline
import matplotlib.pyplot as plt
for i in range(10):
    plt.subplot(1,10,i+1)
    plt.imshow(train[i,:784].reshape(28,28), cmap='Greys_r')
    plt.axis('off')
plt.show()
print('label: %s' % (train_labels[0:10],))

In [ ]:
import numpy as np
import operator

#disable warnings for large float numbers of estimated probabilities(softmax)
np.seterr(all='ignore')

class Perceptron:

    #Activation functions plus Softmax
    def tanh(np_array):
        #return np.tanh(np_array)
        return np.divide((np.subtract(np.exp(np_array),np.exp((-np_array)))),(np.add(np.exp(np_array),np.exp((-np_array))))) 

    def tanh_gradient(np_array):
        return (1 - np.power(Perceptron.tanh(np_array), 2))

    def softplus(np_array):
        return np.log(np.add(1,np.exp(np_array)))

    def softplus_gradient(np_array):
        return np.divide(1,(np.add(1,np.exp(-np_array))))

    def cosine(np_array):
        return np.cos(np_array)

    def cosine_gradient(np_array):
        return -np.sin(np_array)

    def softmax(np_array):
        return np.divide(np.exp(np_array), np.sum(np.exp(np_array),axis = 1,keepdims = True))



    #Dictionary of activation functions
    activation_func = {
        "tanh": lambda np_array: Perceptron.tanh(np_array),
        "softplus": lambda np_array: Perceptron.softplus(np_array),
        "cosine": lambda np_array: Perceptron.cosine(np_array)
    }

    #Dictionary of gradients of activation functions
    activation_grad_func = {
        "tanh": lambda np_array: Perceptron.tanh_gradient(np_array),
        "softplus": lambda np_array: Perceptron.softplus_gradient(np_array),
        "cosine": lambda np_array: Perceptron.cosine_gradient(np_array)
    }
    
    def __init__(self, hidden_layer,epochs,req_term,activation,X,y):
        
        self.hidden_layer = hidden_layer
        self.epochs = epochs
        self.req_term = req_term
        self.activation = activation
        self.X = X
        self.y = y
        self.categories = len(np.unique(self.y))
        self.w1,self.b1,self.w2,self.b2 =  None,None,None,None
        self.dw1,self.db1,self.dw2,self.db2 =  None,None,None,None
        self.a2,self.z2,self.probs = None,None,None
        self.labels = self.labels_onehotvector()
        self.alpha = 1
        self.cost_function_history = []
        self.train_size = self.X.shape[0] 
       
    #Check user's input for pre-defined activation functions
    activation = property(operator.attrgetter('_activation'))   
    @activation.setter
    def activation(self, act):
        if act.lower() not in["tanh","softplus","cosine"]:
            raise Exception("activation must be either 'tanh' or 'softplus' or 'cosine'")
        self._activation = act
        
    
    def weight_init(self):
    
        '''Random initialization of weights using univariate “normal” (Gaussian) distribution 
        of mean 0 and variance 1 divided by square root of input size
        '''
    
        #set the seed for reproducibility
        np.random.seed(1234)
        self.w1 = np.random.randn(self.X.shape[1], self.hidden_layer) / np.sqrt(self.X.shape[1])
        self.b1 = np.zeros((1, self.hidden_layer))
        self.w2 = np.random.randn(self.hidden_layer,self.categories) / np.sqrt(self.X.shape[1])
        self.b2 = np.zeros((1,self.categories))
        
    
    def labels_onehotvector(self):
        
        #convert output labels to one hot vectors
        labels = np.zeros((self.y.shape[0],self.categories))
        for idx in range(labels.shape[0]):
            for jdx in range(self.categories):
                if self.y[idx] == jdx:
                    labels[idx,jdx] = 1
                    break
        return labels
        
    
    
    def forprob(self):
        
        #foreward propagation of the neural network
        self.z2 = self.X.dot(self.w1) + self.b1
        self.a2 = Perceptron.activation_func[self.activation](self.z2)
        z3 = self.a2.dot(self.w2) + self.b2
        self.probs = Perceptron.softmax(z3)
        self.cost_function()
        
     
    def gradcheck(self):
        
        #Use numerical gradient to verify correctness of our gradients implementation
        print("Numerical gradient checking may take several time")
        self.weight_init()
        init_grads = np.concatenate([self.w1.flatten(),self.b1.flatten(),self.w2.flatten(),self.b2.flatten()])
       
        numgrads = np.zeros(init_grads.shape[0])
        perturb = np.zeros(init_grads.shape[0])
        e = 1E-4
        for grad in range(init_grads.shape[0]):
            perturb[grad] = e
            loss1 =self.numgrad_cost(init_grads - perturb) 
            loss2 = self.numgrad_cost(init_grads + perturb) 
            numgrads[grad] = (loss2 - loss1) / (2*e) 
            perturb[grad] = 0
        self.forprob()
        self.backprob()
        updated_grads = np.concatenate([self.dw1.flatten(),self.db1.flatten(),self.dw2.flatten(),self.db2.flatten()])
        diff = updated_grads  - numgrads
        max_grad_diff = np.absolute(diff).max()
        print("Maximum absolute difference between gradients and numerical gradients is:",max_grad_diff)
        print("If the above value is really small(<1e-6), then backpropagation is implemented correctly")
        
        
    def numgrad_cost(self,grads):
        
        #Compute cost function for numerical gradients
        #Unroll gradients
        
        w1 = grads[:self.X.shape[1]*self.hidden_layer].reshape(self.X.shape[1],self.hidden_layer)
        b1 = grads[self.X.shape[1]*self.hidden_layer:(self.X.shape[1]*self.hidden_layer)+self.hidden_layer].reshape(1,self.hidden_layer)
        w2 = grads[(self.X.shape[1]*self.hidden_layer)+self.hidden_layer:(self.X.shape[1]*self.hidden_layer)+self.hidden_layer+(self.hidden_layer*self.categories)].reshape(self.hidden_layer,self.categories)
        b2 =grads[(self.X.shape[1]*self.hidden_layer)+self.hidden_layer+(self.hidden_layer*self.categories):].reshape(1,self.categories)
        #foreward propagation
        z2 = self.X.dot(w1) + b1
        a2 = Perceptron.activation_func[self.activation](z2)
        z3 = a2.dot(w2) + b2
        probs = Perceptron.softmax(z3)
        log_probs = np.multiply(np.log(probs),self.labels)
        cost = log_probs.sum()
        cost-= self.req_term/2 * (np.sum(np.square(w1)) + np.sum(np.square(w2)))
        return cost
        
    def backprob(self):
        
        #back propagation of the neural network 
        delta3 = np.subtract(self.labels,self.probs)
        self.dw2 = (self.a2.T).dot(delta3)
        self.db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(self.w2.T) * Perceptron.activation_grad_func[self.activation](self.z2)
        self.dw1 = np.dot(self.X.T, delta2)
        self.db1 = np.sum(delta2, axis=0)
 
        #Subtract regularization terms for w1,w2
        self.dw2 -= self.req_term *  self.w2
        self.dw1 -= self.req_term *  self.w1
        
        # Weights update
        self.w1 += self.alpha *  self.dw1 / self.train_size
        self.b1 += self.alpha * self.db1 / self.train_size
        self.w2 += self.alpha * self.dw2 / self.train_size
        self.b2 += self.alpha * self.db2 / self.train_size
        
        
        
    def cost_function(self):
        
        #calculate max log likelihood plus reqularization term as cost function
        log_probs = np.multiply(np.log(self.probs),self.labels)
        
        cost = log_probs.sum()
        cost-= self.req_term/2 * (np.sum(np.square(self.w1)) + np.sum(np.square(self.w2)))
        self.cost_function_history.append(cost)
        
    def fit(self,verbose = False):
        
        #train neural network
        self.weight_init()
        for epoch in range(self.epochs):
            self.forprob()
            if (epoch % 20 == 0):
                self.alpha = 1
            if ((epoch > 1) and (self.cost_function_history[-2]) <= self.cost_function_history[-1]):
                self.alpha =  round(self.alpha/1.1,2)
            self.backprob()
            if(verbose and epoch % 10 == 0):
                print("Cost funcion after iteration %i: %f" %(epoch,self.cost_function_history[-1]))
           
    def predict(self,test_data,test_labels):
        
        print("Hidden Layer size: %i Activation function: %s lamda: %s" %(self.hidden_layer,self.activation,self.req_term))
        print("Cost Function value: %f" %(self.cost_function_history[-1]))
        train_preds = np.argmax(self.probs,axis = 1)
        train_error = 1.0 - np.sum(train_preds == self.y)/self.train_size
        print("Error on train data:", train_error )
        
        #Predict on test_data using learned parameters w1,b1,w2,b2
        z1 = test_data.dot(self.w1) + self.b1
        a1 = Perceptron.activation_func[self.activation](z1)
        z2 = a1.dot(self.w2) + self.b2
        probs = Perceptron.softmax(z2)
        test_preds = np.argmax(probs, axis=1)
        test_error = 1.0 - np.sum(test_preds == test_labels)/test_data.shape[0]
        print("Error on test data:", test_error )

# nn_gradcheck = Perceptron(10,2500,0.00,"tanh",train_data[:2000,:],train_labels[:2000])
# nn_gradcheck.gradcheck()       

nn = Perceptron(100,100,0.00,"tanh",train_data,train_labels)
nn.fit(True)
nn.predict(test_data,test_labels)